In [1]:
import requests
from bs4 import BeautifulSoup
import os
import re
import pandas as pd

# URL de la página
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Realizar la solicitud GET a la página
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Crear una carpeta para almacenar los archivos descargados
os.makedirs('parquet_files', exist_ok=True)
os.makedirs('transformed_files', exist_ok=True)

# Función para verificar si el archivo está dentro del rango de años
def is_within_year_range(file_name, start_year, end_year):
    match = re.search(r'(\d{4})', file_name)
    if match:
        year = int(match.group(1))
        return start_year >= year >= end_year
    return False

# Rango de años
start_year = 2023
end_year = 2019

# Función para extraer el año y el mes del nombre del archivo
def get_year_month_from_filename(file_name):
    match = re.search(r'(\d{4})-(\d{2})', file_name)
    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        return year, month
    return None, None

# Función para transformar los datos
def transform_data(df, file_name):
    year, month = get_year_month_from_filename(file_name)
    if year is None or month is None:
        return pd.DataFrame()  # Devuelve un DataFrame vacío si no se puede extraer el año y el mes

    if 'green_tripdata' in file_name:
        columns_to_keep = ['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'trip_distance', 'fare_amount', 'PULocationID']
        df = df[columns_to_keep]
        df['Es_verde'] = 1
        # Separar fecha y hora
        df['Fecha'] = pd.to_datetime(df['lpep_pickup_datetime']).dt.date
        df['pickup_time'] = pd.to_datetime(df['lpep_pickup_datetime']).dt.time
        df['dropoff_time'] = pd.to_datetime(df['lpep_dropoff_datetime']).dt.time
        df.drop(columns=['lpep_pickup_datetime', 'lpep_dropoff_datetime'], inplace=True)
    if 'yellow_tripdata' in file_name:
        columns_to_keep = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'fare_amount', 'PULocationID']
        df = df[columns_to_keep]
        df['Es_verde'] = 0
        # Separar fecha y hora
        df['Fecha'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.date
        df['pickup_time'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.time
        df['dropoff_time'] = pd.to_datetime(df['tpep_dropoff_datetime']).dt.time
        df.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime'], inplace=True)

    # Filtrar filas por el año y mes
    df = df[(pd.to_datetime(df['Fecha']).dt.year == year) & (pd.to_datetime(df['Fecha']).dt.month == month)]
    
    # Filtrar filas con PULocationID
    df = df[(df['PULocationID'] != 1) & (df['PULocationID'] != 264) & (df['PULocationID'] != 265)]

    return df

# Buscar todos los enlaces de archivos Parquet
for link in soup.find_all('a', href=True):
    href = link['href']
    if href.endswith('.parquet'):
        file_name = href.split('/')[-1]
        # Filtrar solo archivos que contengan 'green_tripdata' o 'yellow_tripdata' y estén en el rango de años
        if ('green_tripdata' in file_name or 'yellow_tripdata' in file_name) and is_within_year_range(file_name, start_year, end_year):
            # Descargar el archivo
            file_url = href
            file_response = requests.get(file_url)
            parquet_path = os.path.join('parquet_files', file_name)
            with open(parquet_path, 'wb') as file:
                file.write(file_response.content)
            
            print(f"Descargado: {file_name}")
            
            # Leer el archivo .parquet
            df = pd.read_parquet(parquet_path)
        
            # Transformar los datos
            transformed_df = transform_data(df, file_name)
            
            # Guardar el archivo transformado
            transformed_file_name = f"transformed_{file_name.replace('.parquet', '.csv')}"
            transformed_path = os.path.join('transformed_files', transformed_file_name)
            transformed_df.to_csv(transformed_path, index=False)
            
            print(f"Transformado y guardado: {transformed_file_name}")

print("Descarga y transformación completadas.")


Descargado: yellow_tripdata_2023-01.parquet


C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Es_verde'] = 0
C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.date
C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

Transformado y guardado: transformed_yellow_tripdata_2023-01.csv
Descargado: green_tripdata_2023-01.parquet


C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Es_verde'] = 1
C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha'] = pd.to_datetime(df['lpep_pickup_datetime']).dt.date
C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

Transformado y guardado: transformed_green_tripdata_2023-01.csv
Descargado: yellow_tripdata_2023-02.parquet


...

Transformado y guardado: transformed_green_tripdata_2019-11.csv
Descargado: yellow_tripdata_2019-12.parquet


C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Es_verde'] = 0
C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.date
C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

Transformado y guardado: transformed_yellow_tripdata_2019-12.csv
Descargado: green_tripdata_2019-12.parquet


C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Es_verde'] = 1
C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha'] = pd.to_datetime(df['lpep_pickup_datetime']).dt.date
C:\Users\aje0v\AppData\Local\Temp\ipykernel_2384\1274744233.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

Transformado y guardado: transformed_green_tripdata_2019-12.csv
Descarga y transformación completadas.
